In [10]:
import pandas as pd
import psycopg2
import pyodbc
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import text

## Redshift Connection

In [11]:
## Redshift connection parameters
red_endpoint = 'orinoco.c2qsr3tflm58.us-east-1.redshift.amazonaws.com'
##user = 'mrodrigues'
password = 'xxx'
red_port = 5439
red_dbname = 'orinoco'

In [12]:
## Redshift SQL engine
red_engine_string ="postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (user, password, red_endpoint, red_port, red_dbname)
red_engine = create_engine(red_engine_string)

## RDS Connection

In [13]:
## RDS connection parameters
rds_endpoint = 'live.letsgetchecked-ds.com'
##user = 'mrodrigues'
password = 'xxx'
rds_port = 1433
rds_dbname = 'use_core'

In [14]:
## RDS connection string
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+rds_endpoint+';DATABASE='+rds_dbname+';UID='+user+';PWD='+ password)

## Importation of the DAL Shipment table from Redshift

In [4]:
## to import the table DAL Shipment to a dataframe
red_df1_part1 = pd.read_sql_query(text("select * from dal_us.bi_shipment where order_created_at_outbound >= '2021-01-01' and order_created_at_outbound <= current_date - interval '1 DAY' union select * from dal_eu.bi_shipment where order_created_at_outbound >= '2021-01-01' and order_created_at_outbound <= current_date - interval '1 DAY' order by bi_shipment_id ASC"), red_engine)
red_df1_part1.head()

,bi_shipment_id,reseller_name,order_id,order_actual_dispatch_time,order_created_at_outbound,barcoded_id,test_kit_name,shipment_id_outbound,shipment_provider_name_outbound,shipment_level_outbound,...,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound,payer_program_pretty_name,created_at,updated_at
0,100000491881_104005036,Optum Housecalls,100941545,2021-10-25 13:00:59.620,2021-10-23 10:19:14.003,100000491881,Hep C,104005036,USPS,First,...,UPS,Next Day Air,1ZR866X7Y004547559,NaT,2021-10-28 15:45:00,2021-11-12 18:43:39,2021-11-13 11:58:51,OptumHepCUHCHouseCalls2021,2022-01-20 18:16:47,NaT
1,100000540030_104644900,www.letsgetchecked.com,101092131,2022-01-03 13:39:43.133,2022-01-02 22:20:27.593,100000540030,Standard 5,104644900,UPSMailInnovations,Expedited Mail Innovations,...,UPS,Next Day Air,1ZR866X7Y007435469,NaT,NaT,2022-01-10 16:51:46,2022-01-12 10:26:13,None,2022-01-20 18:16:47,2022-01-20 20:16:39
2,100005562278_103816477,Humana,100885734,2021-10-07 18:56:59.170,2021-10-06 21:20:37.187,100005562278,COVID-19 Home Test (PCR Lower Nasal),103816477,UPS,UPS Next Day Air,...,UPS,Next Day Air,1ZR866X7Y000805161,2021-10-07 16:34:30,2021-10-08 10:16:31,NaT,NaT,HUMANACOVIDDTC2021,2022-01-20 18:16:47,NaT
3,100005567616_104531022,TECHOPS US WALKTHROUGH,101050724,2021-12-14 19:18:10.640,2021-12-14 16:30:33.167,100005567616,COVID-19 Home Test (PCR Lower Nasal),104531022,UPSMailInnovations,Expedited Mail Innovations,...,UPS,Next Day Air,1Z550F28Y083330586,NaT,NaT,NaT,NaT,DKTechOpsUS2021,2022-01-20 18:16:47,NaT
4,10000644936_103736015,United Health Group,100850474,2021-10-11 16:23:26.190,2021-09-29 21:28:42.980,10000644936,Kidney Health Evaluation,103736015,USPS,First,...,UPS,Next Day Air,1ZR866X7Y004328045,NaT,2021-10-15 14:38:00,NaT,NaT,UHCMEDICAREKED2021,2022-01-20 18:16:47,NaT


In [5]:
## to see the total of records and columns
red_df1_part1.shape

(793239, 23)

In [6]:
## to import the table DAL Shipment materialized view to a dataframe - part2
red_df1_part2 = pd.read_sql_query(text("select * from dal_us.bi_shipment where order_created_at_outbound < '2021-01-01' union select * from dal_eu.bi_shipment where order_created_at_outbound < '2021-01-01' order by bi_shipment_id ASC"), red_engine)
red_df1_part2.head() 

,bi_shipment_id,reseller_name,order_id,order_actual_dispatch_time,order_created_at_outbound,barcoded_id,test_kit_name,shipment_id_outbound,shipment_provider_name_outbound,shipment_level_outbound,...,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound,payer_program_pretty_name,created_at,updated_at
0,100003742755_101391128,Fidelity,100292869,2020-11-18 15:26:46.320,2020-11-18 11:28:06.690,100003742755,COVID-19 Home Test (PCR Lower Nasal),101391128,UPS,UPS Next Day Air,...,UPS,Next Day Air,1Z550F28Y092170518,2020-11-18 20:46:16,2020-11-19 09:36:35,NaT,2020-11-21 10:07:30,FIDELITYCOVID19,2022-01-20 18:16:47,NaT
1,100004370467_101893808,www.letsgetchecked.com,100386844,2020-12-26 15:36:50.397,2020-12-26 09:20:36.823,100004370467,COVID-19 Home Test (PCR Lower Nasal),101893808,UPS,UPS Next Day Air,...,UPS,Next Day Air,1Z550F28Y011699832,2020-12-28 19:16:09,2020-12-29 14:17:21,NaT,NaT,None,2022-01-20 18:16:47,NaT
2,100004472400_101499810,BS - Yellowstone Club Pre-arrival,100343660,2020-12-03 23:34:58.033,2020-12-03 15:42:43.100,100004472400,COVID-19 (PCR Lower Nasal),101499810,UPS,UPS Next Day Air,...,UPS,Next Day Air,1Z550F28Y013558141,2020-12-03 19:49:01,2020-12-04 12:48:55,2020-12-21 16:21:32,2020-12-22 08:33:28,BSYELLOWSTONEPREARRIVALCOVID,2022-01-20 18:16:47,NaT
3,100009629350_101489232,United Health Group,100333692,2020-12-07 22:41:30.120,2020-12-03 10:51:58.227,100009629350,Hep C,101489232,USPS,First,...,UPS,Next Day Air,1Z550F28Y089964828,NaT,2020-12-11 10:58:00,NaT,NaT,UHCMEDICAREHEPC,2022-01-20 18:16:47,NaT
4,100011905254_77421,www.letsgetchecked.com,99908,2019-12-03 11:26:33.453,2019-12-02 18:29:00.997,100011905254,Simple 2,77421,AnPost,An psot,...,None,None,None,NaT,NaT,NaT,NaT,None,2022-01-20 18:06:51,NaT


In [7]:
## to see the total of records and columns
red_df1_part2.shape

(535962, 23)

In [8]:
## to append the 2 dataframes into 1 - Redshift ##
red_df1 = red_df1_part2.append(red_df1_part1, ignore_index=False)
red_df1.head()

,bi_shipment_id,reseller_name,order_id,order_actual_dispatch_time,order_created_at_outbound,barcoded_id,test_kit_name,shipment_id_outbound,shipment_provider_name_outbound,shipment_level_outbound,...,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound,payer_program_pretty_name,created_at,updated_at
0,100003742755_101391128,Fidelity,100292869,2020-11-18 15:26:46.320,2020-11-18 11:28:06.690,100003742755,COVID-19 Home Test (PCR Lower Nasal),101391128,UPS,UPS Next Day Air,...,UPS,Next Day Air,1Z550F28Y092170518,2020-11-18 20:46:16,2020-11-19 09:36:35,NaT,2020-11-21 10:07:30,FIDELITYCOVID19,2022-01-20 18:16:47,NaT
1,100004370467_101893808,www.letsgetchecked.com,100386844,2020-12-26 15:36:50.397,2020-12-26 09:20:36.823,100004370467,COVID-19 Home Test (PCR Lower Nasal),101893808,UPS,UPS Next Day Air,...,UPS,Next Day Air,1Z550F28Y011699832,2020-12-28 19:16:09,2020-12-29 14:17:21,NaT,NaT,None,2022-01-20 18:16:47,NaT
2,100004472400_101499810,BS - Yellowstone Club Pre-arrival,100343660,2020-12-03 23:34:58.033,2020-12-03 15:42:43.100,100004472400,COVID-19 (PCR Lower Nasal),101499810,UPS,UPS Next Day Air,...,UPS,Next Day Air,1Z550F28Y013558141,2020-12-03 19:49:01,2020-12-04 12:48:55,2020-12-21 16:21:32,2020-12-22 08:33:28,BSYELLOWSTONEPREARRIVALCOVID,2022-01-20 18:16:47,NaT
3,100009629350_101489232,United Health Group,100333692,2020-12-07 22:41:30.120,2020-12-03 10:51:58.227,100009629350,Hep C,101489232,USPS,First,...,UPS,Next Day Air,1Z550F28Y089964828,NaT,2020-12-11 10:58:00,NaT,NaT,UHCMEDICAREHEPC,2022-01-20 18:16:47,NaT
4,100011905254_77421,www.letsgetchecked.com,99908,2019-12-03 11:26:33.453,2019-12-02 18:29:00.997,100011905254,Simple 2,77421,AnPost,An psot,...,None,None,None,NaT,NaT,NaT,NaT,None,2022-01-20 18:06:51,NaT


## Understand basic information about the Redshift dataframe

In [9]:
## to see the total of records and columns
red_df1.shape

(1329201, 23)

In [16]:
## to see the basic information about the dataframe
red_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1326836 entries, 0 to 790873
Data columns (total 23 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   bi_shipment_id                     1326836 non-null  object        
 1   reseller_name                      1326836 non-null  object        
 2   order_id                           1326836 non-null  int64         
 3   order_actual_dispatch_time         1323551 non-null  datetime64[ns]
 4   order_created_at_outbound          1326836 non-null  datetime64[ns]
 5   barcoded_id                        1326836 non-null  int64         
 6   test_kit_name                      1326836 non-null  object        
 7   shipment_id_outbound               1326836 non-null  int64         
 8   shipment_provider_name_outbound    1326836 non-null  object        
 9   shipment_level_outbound            1326836 non-null  object        
 10  shipmen

In [17]:
## to detect null values in the dataframe
red_df1.isnull().sum()

bi_shipment_id                             0
reseller_name                              0
order_id                                   0
order_actual_dispatch_time              3285
order_created_at_outbound                  0
barcoded_id                                0
test_kit_name                              0
shipment_id_outbound                       0
shipment_provider_name_outbound            0
shipment_level_outbound                    0
shipment_tracking_number_outbound          0
shipment_id_inbound                   265515
order_created_at_inbound              265515
shipment_provider_name_inbound        265515
shipment_level_inbound                265515
shipment_tracking_number_inbound      265515
shipment_pickup_time_outbound         270477
shipment_delivery_time_outbound       329566
shipment_pickup_time_inbound          841334
shipment_delivery_time_inbound        872975
payer_program_pretty_name             602305
created_at                                 0
updated_at

## Importation of the BI team query output from RDS

In [15]:
## to import the query of BI Team in a dataframe
rds_sql1_part1 = """
WITH single_kit_orders AS (select
o.order_id,
count(oi.order_item_id) as count_kits
from
use_core.dbo.[order] o
inner join use_core.dbo.order_item oi on o.order_id = oi.order_id
group by o.order_id
having count(oi.order_item_id) = 1
UNION
select
o.order_id,
count(oi.order_item_id) as count_kits
from
euw_core.dbo.[order] o
inner join euw_core.dbo.order_item oi on o.order_id = oi.order_id
group by o.order_id
having count(oi.order_item_id) = 1),

outbound AS (select 
re.[name] as reseller_name
,p.pretty_name as payer_program_pretty_name
,o.order_id
,o.dta000 as order_created_at_outbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_outbound
,s.[level] as shipment_level_outbound
,tk.[name] as test_kit_name
,o.actual_dispatch_time as order_actual_dispatch_time
,s.shipment_id as shipment_id_outbound
,s.tracking_number as shipment_tracking_number_outbound
,s.pickup_time as shipment_pickup_time_outbound
,s.delivery_time as shipment_delivery_time_outbound
,'USE' as region
from 
[use_core].[dbo].[order] o 
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [use_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [use_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [use_core].[dbo].[batch] ba ON bi.[batch_id] = ba.[batch_id]
INNER JOIN [use_core].[dbo].[order_shipment] os ON os.order_id= o.order_id
INNER JOIN [use_core].[dbo].[shipment] s ON s.shipment_id = os.shipment_id and s.direction = 0
INNER JOIN [use_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
LEFT JOIN [use_core].[dbo].[test_kit] AS tk  ON tk.test_kit_id = oi.test_kit_id
LEFT JOIN [use_core].[dbo].[reseller] AS re ON re.reseller_id = ba.reseller_id
LEFT JOIN [use_core].[dbo].[opt_in] opt ON o.order_id = opt.order_id
LEFT JOIN [use_core].[dbo].[payer_program_instance] ins ON ins.program_instance_id = opt.program_instance_id
LEFT JOIN [use_core].[dbo].[payer_program] p ON ins.program_id = p.program_id
where o.[is_internal] = 0

UNION
select 
re.[name] as reseller_name
,p.pretty_name as payer_program_pretty_name
,o.order_id
,o.dta000 as order_created_at_outbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_outbound
,s.[level] as shipment_level_outbound
,tk.[name] as test_kit_name
,o.actual_dispatch_time as order_actual_dispatch_time
,s.shipment_id as shipment_id_outbound
,s.tracking_number as shipment_tracking_number_outbound
,s.pickup_time as shipment_delivery_time_outbound
,s.delivery_time as shipment_delivery_time_outbound
,'EUW' as region
from 
[euw_core].[dbo].[order] o 
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [euw_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [euw_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [euw_core].[dbo].[batch] ba ON bi.[batch_id] = ba.[batch_id]
INNER JOIN [euw_core].[dbo].[order_shipment] os ON os.order_id= o.order_id
INNER JOIN [euw_core].[dbo].[shipment] s ON s.shipment_id = os.shipment_id and s.direction = 0
INNER JOIN [euw_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
LEFT JOIN [euw_core].[dbo].[test_kit] AS tk  ON tk.test_kit_id = oi.test_kit_id
LEFT JOIN [euw_core].[dbo].[reseller] AS re ON re.reseller_id = ba.reseller_id
LEFT JOIN [euw_core].[dbo].[opt_in] opt ON o.order_id = opt.order_id
LEFT JOIN [euw_core].[dbo].[payer_program_instance] ins ON ins.program_instance_id = opt.program_instance_id
LEFT JOIN [euw_core].[dbo].[payer_program] p ON ins.program_id = p.program_id
where o.[is_internal] = 0),

inbound AS (select
o.order_id
,o.dta000 as order_created_at_inbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_inbound
,s.shipment_id as shipment_id_inbound
,s.[level] as shipment_level_inbound
,s.tracking_number as shipment_tracking_number_inbound
,s.pickup_time as shipment_pickup_time_inbound
,s.delivery_time as shipment_delivery_time_inbound
,'USE' as region
from [use_core].[dbo].[order] o
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [use_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [use_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [use_core].[dbo].[shipment] s ON s.shipment_id = bi.shipment_id 
INNER JOIN [use_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
where o.[is_internal] = 0

UNION
select 
o.order_id
,o.dta000 as order_created_at_inbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_inbound
,s.shipment_id as shipment_id_inbound
,s.[level] as shipment_level_inbound
,s.tracking_number as shipment_tracking_number_inbound
,s.pickup_time as shipment_pickup_time_inbound
,s.delivery_time as shipment_delivery_time_inbound
,'EUW' as region
from [euw_core].[dbo].[order] o
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [euw_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [euw_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [euw_core].[dbo].[shipment] s ON s.shipment_id = bi.shipment_id 
INNER JOIN [euw_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
where o.[is_internal] = 0)

select CONCAT(CONCAT(COALESCE(outbound.barcoded_id,-1),'_'),COALESCE(outbound.shipment_id_outbound,-1)) as bi_shipment_id
,outbound.order_created_at_outbound
,outbound.reseller_name
,outbound.payer_program_pretty_name
,outbound.region
,outbound.order_id
,outbound.barcoded_id
,outbound.shipment_id_outbound
,outbound.test_kit_name
,outbound.order_actual_dispatch_time
,outbound.shipment_provider_name_outbound
,outbound.shipment_level_outbound
,outbound.shipment_tracking_number_outbound
,inbound.order_created_at_inbound
,inbound.shipment_provider_name_inbound
,inbound.shipment_level_inbound
,inbound.shipment_tracking_number_inbound
,inbound.shipment_id_inbound
,outbound.shipment_pickup_time_outbound
,outbound.shipment_delivery_time_outbound
,inbound.shipment_pickup_time_inbound
,inbound.shipment_delivery_time_inbound
from outbound
left join inbound
ON outbound.barcoded_id = inbound.barcoded_id AND outbound.region = inbound.region
where convert(date,outbound.order_created_at_outbound) >= '2021-01-01' and
	convert(date,outbound.order_created_at_outbound) <= dateadd(dd,-2,cast(getdate() as date))
order by CONCAT(CONCAT(COALESCE(outbound.barcoded_id,-1),'_'),COALESCE(outbound.shipment_id_outbound,-1)) ASC
"""
rds_df1_part1 = pd.read_sql_query(rds_sql1_part1, cnxn)
rds_df1_part1.head()

,bi_shipment_id,order_created_at_outbound,reseller_name,payer_program_pretty_name,region,order_id,barcoded_id,shipment_id_outbound,test_kit_name,order_actual_dispatch_time,...,shipment_tracking_number_outbound,order_created_at_inbound,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_id_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound
0,100000491881_104005036,2021-10-23 10:19:14.003,Optum Housecalls,OptumHepCUHCHouseCalls2021,USE,100941545,100000491881,104005036,Hep C,2021-10-25 13:00:59.620,...,9400111109160801336257,2021-10-23 10:19:14.003,UPS,Next Day Air,1ZR866X7Y004547559,103860807.0,NaT,2021-10-28 15:45:00,2021-11-12 18:43:39,2021-11-13 11:58:51
1,100000540030_104644900,2022-01-02 22:20:27.593,www.letsgetchecked.com,None,USE,101092131,100000540030,104644900,Standard 5,2022-01-03 13:39:43.133,...,92748902711971543400168390,2022-01-02 22:20:27.593,UPS,Next Day Air,1ZR866X7Y007435469,104366797.0,NaT,NaT,2022-01-10 16:51:46,2022-01-12 10:26:13
2,100005562278_103816477,2021-10-06 21:20:37.187,Humana,HUMANACOVIDDTC2021,USE,100885734,100005562278,103816477,COVID-19 Home Test (PCR Lower Nasal),2021-10-07 18:56:59.170,...,1ZWV28010137334562,2021-10-06 21:20:37.187,UPS,Next Day Air,1ZR866X7Y000805161,102916958.0,2021-10-07 16:34:30,2021-10-08 10:16:31,NaT,NaT
3,100005567616_104531022,2021-12-14 16:30:33.167,TECHOPS US WALKTHROUGH,DKTechOpsUS2021,USE,101050724,100005567616,104531022,COVID-19 Home Test (PCR Lower Nasal),2021-12-14 19:18:10.640,...,92748902711971543475160916,2021-12-14 16:30:33.167,UPS,Next Day Air,1Z550F28Y083330586,102515491.0,NaT,NaT,NaT,NaT
4,10000644936_103736015,2021-09-29 21:28:42.980,United Health Group,UHCMEDICAREKED2021,USE,100850474,10000644936,103736015,Kidney Health Evaluation,2021-10-11 16:23:26.190,...,9400111109160806737974,2021-09-29 21:28:42.980,UPS,Next Day Air,1ZR866X7Y004328045,103633437.0,NaT,2021-10-15 14:38:00,NaT,NaT


In [16]:
## to see the total of records and columns
rds_df1_part1.shape

(793239, 22)

In [17]:
## to import the query of BI Team in a dataframe - part2
rds_sql1_part2 = """
WITH single_kit_orders AS (select
o.order_id,
count(oi.order_item_id) as count_kits
from
use_core.dbo.[order] o
inner join use_core.dbo.order_item oi on o.order_id = oi.order_id
group by o.order_id
having count(oi.order_item_id) = 1
UNION
select
o.order_id,
count(oi.order_item_id) as count_kits
from
euw_core.dbo.[order] o
inner join euw_core.dbo.order_item oi on o.order_id = oi.order_id
group by o.order_id
having count(oi.order_item_id) = 1),

outbound AS (select
re.[name] as reseller_name
,p.pretty_name as payer_program_pretty_name
,o.order_id
,o.dta000 as order_created_at_outbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_outbound
,s.[level] as shipment_level_outbound
,tk.[name] as test_kit_name
,o.actual_dispatch_time as order_actual_dispatch_time
,s.shipment_id as shipment_id_outbound
,s.tracking_number as shipment_tracking_number_outbound
,s.pickup_time as shipment_pickup_time_outbound
,s.delivery_time as shipment_delivery_time_outbound
,'USE' as region
from
[use_core].[dbo].[order] o
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [use_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [use_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [use_core].[dbo].[batch] ba ON bi.[batch_id] = ba.[batch_id]
INNER JOIN [use_core].[dbo].[order_shipment] os ON os.order_id= o.order_id
INNER JOIN [use_core].[dbo].[shipment] s ON s.shipment_id = os.shipment_id and s.direction = 0
INNER JOIN [use_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
LEFT JOIN [use_core].[dbo].[test_kit] AS tk  ON tk.test_kit_id = oi.test_kit_id
LEFT JOIN [use_core].[dbo].[reseller] AS re ON re.reseller_id = ba.reseller_id
LEFT JOIN [use_core].[dbo].[opt_in] opt ON o.order_id = opt.order_id
LEFT JOIN [use_core].[dbo].[payer_program_instance] ins ON ins.program_instance_id = opt.program_instance_id
LEFT JOIN [use_core].[dbo].[payer_program] p ON ins.program_id = p.program_id
where o.[is_internal] = 0

UNION
select
re.[name] as reseller_name
,p.pretty_name as payer_program_pretty_name
,o.order_id
,o.dta000 as order_created_at_outbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_outbound
,s.[level] as shipment_level_outbound
,tk.[name] as test_kit_name
,o.actual_dispatch_time as order_actual_dispatch_time
,s.shipment_id as shipment_id_outbound
,s.tracking_number as shipment_tracking_number_outbound
,s.pickup_time as shipment_delivery_time_outbound
,s.delivery_time as shipment_delivery_time_outbound
,'EUW' as region
from
[euw_core].[dbo].[order] o
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [euw_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [euw_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [euw_core].[dbo].[batch] ba ON bi.[batch_id] = ba.[batch_id]
INNER JOIN [euw_core].[dbo].[order_shipment] os ON os.order_id= o.order_id
INNER JOIN [euw_core].[dbo].[shipment] s ON s.shipment_id = os.shipment_id and s.direction = 0
INNER JOIN [euw_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
LEFT JOIN [euw_core].[dbo].[test_kit] AS tk  ON tk.test_kit_id = oi.test_kit_id
LEFT JOIN [euw_core].[dbo].[reseller] AS re ON re.reseller_id = ba.reseller_id
LEFT JOIN [euw_core].[dbo].[opt_in] opt ON o.order_id = opt.order_id
LEFT JOIN [euw_core].[dbo].[payer_program_instance] ins ON ins.program_instance_id = opt.program_instance_id
LEFT JOIN [euw_core].[dbo].[payer_program] p ON ins.program_id = p.program_id
where o.[is_internal] = 0),

inbound AS (select
o.order_id
,o.dta000 as order_created_at_inbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_inbound
,s.shipment_id as shipment_id_inbound
,s.[level] as shipment_level_inbound
,s.tracking_number as shipment_tracking_number_inbound
,s.pickup_time as shipment_pickup_time_inbound
,s.delivery_time as shipment_delivery_time_inbound
,'USE' as region
from [use_core].[dbo].[order] o
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [use_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [use_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [use_core].[dbo].[shipment] s ON s.shipment_id = bi.shipment_id
INNER JOIN [use_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
where o.[is_internal] = 0

UNION
select
o.order_id
,o.dta000 as order_created_at_inbound
,bi.barcoded_id
,sp.[name] as shipment_provider_name_inbound
,s.shipment_id as shipment_id_inbound
,s.[level] as shipment_level_inbound
,s.tracking_number as shipment_tracking_number_inbound
,s.pickup_time as shipment_pickup_time_inbound
,s.delivery_time as shipment_delivery_time_inbound
,'EUW' as region
from [euw_core].[dbo].[order] o
INNER JOIN single_kit_orders sko ON sko.order_id = o.order_id
INNER JOIN [euw_core].[dbo].[order_item] oi ON o.order_id = oi.order_id
INNER JOIN [euw_core].[dbo].[batch_item] bi ON bi.[batch_item_id] = oi.[batch_item_id]
INNER JOIN [euw_core].[dbo].[shipment] s ON s.shipment_id = bi.shipment_id
INNER JOIN [euw_core].[dbo].[dimension_shipment_provider] sp on s.provider_id = sp.[value]
where o.[is_internal] = 0)

select
CONCAT(CONCAT(COALESCE(outbound.barcoded_id,-1),'_'),COALESCE(outbound.shipment_id_outbound,-1)) as bi_shipment_id
,outbound.order_created_at_outbound
,outbound.reseller_name
,outbound.payer_program_pretty_name
,outbound.region
,outbound.order_id
,outbound.barcoded_id
,outbound.shipment_id_outbound
,outbound.test_kit_name
,outbound.order_actual_dispatch_time
,outbound.shipment_provider_name_outbound
,outbound.shipment_level_outbound
,outbound.shipment_tracking_number_outbound
,inbound.order_created_at_inbound
,inbound.shipment_provider_name_inbound
,inbound.shipment_level_inbound
,inbound.shipment_tracking_number_inbound
,inbound.shipment_id_inbound
,outbound.shipment_pickup_time_outbound
,outbound.shipment_delivery_time_outbound
,inbound.shipment_pickup_time_inbound
,inbound.shipment_delivery_time_inbound
from outbound
left join inbound
ON outbound.barcoded_id = inbound.barcoded_id AND outbound.region = inbound.region
where convert(date,outbound.order_created_at_outbound) < '2021-01-01'
order by CONCAT(CONCAT(COALESCE(outbound.barcoded_id,-1),'_'),COALESCE(outbound.shipment_id_outbound,-1)) ASC
"""
rds_df1_part2 = pd.read_sql_query(rds_sql1_part2, cnxn)
rds_df1_part2.head()

,bi_shipment_id,order_created_at_outbound,reseller_name,payer_program_pretty_name,region,order_id,barcoded_id,shipment_id_outbound,test_kit_name,order_actual_dispatch_time,...,shipment_tracking_number_outbound,order_created_at_inbound,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_id_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound
0,100003742755_101391128,2020-11-18 11:28:06.690,Fidelity,FIDELITYCOVID19,USE,100292869,100003742755,101391128,COVID-19 Home Test (PCR Lower Nasal),2020-11-18 15:26:46.320,...,1ZWV28014424426531,2020-11-18 11:28:06.690,UPS,Next Day Air,1Z550F28Y092170518,100914328.0,2020-11-18 20:46:16,2020-11-19 09:36:35,NaT,2020-11-21 10:07:30
1,100004370467_101893808,2020-12-26 09:20:36.823,www.letsgetchecked.com,None,USE,100386844,100004370467,101893808,COVID-19 Home Test (PCR Lower Nasal),2020-12-26 15:36:50.397,...,1ZWV28014432851691,2020-12-26 09:20:36.823,UPS,Next Day Air,1Z550F28Y011699832,101646453.0,2020-12-28 19:16:09,2020-12-29 14:17:21,NaT,NaT
2,100004472400_101499810,2020-12-03 15:42:43.100,BS - Yellowstone Club Pre-arrival,BSYELLOWSTONEPREARRIVALCOVID,USE,100343660,100004472400,101499810,COVID-19 (PCR Lower Nasal),2020-12-03 23:34:58.033,...,1ZWV28014439278914,2020-12-03 15:42:43.100,UPS,Next Day Air,1Z550F28Y013558141,101163535.0,2020-12-03 19:49:01,2020-12-04 12:48:55,2020-12-21 16:21:32,2020-12-22 08:33:28
3,100009629350_101489232,2020-12-03 10:51:58.227,United Health Group,UHCMEDICAREHEPC,USE,100333692,100009629350,101489232,Hep C,2020-12-07 22:41:30.120,...,9400116902881675927238,2020-12-03 10:51:58.227,UPS,Next Day Air,1Z550F28Y089964828,101629583.0,NaT,2020-12-11 10:58:00,NaT,NaT
4,100011905254_77421,2019-12-02 18:29:00.997,www.letsgetchecked.com,None,EUW,99908,100011905254,77421,Simple 2,2019-12-03 11:26:33.453,...,LG044647425IE,NaT,None,None,None,NaN,NaT,NaT,NaT,NaT


In [18]:
## to see the total of records and columns
rds_df1_part2.shape

(535962, 22)

In [19]:
## to append the 2 dataframes into 1 - RDS ##
rds_df1 = rds_df1_part2.append(rds_df1_part1, ignore_index=False)
rds_df1.head()

,bi_shipment_id,order_created_at_outbound,reseller_name,payer_program_pretty_name,region,order_id,barcoded_id,shipment_id_outbound,test_kit_name,order_actual_dispatch_time,...,shipment_tracking_number_outbound,order_created_at_inbound,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_id_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound
0,100003742755_101391128,2020-11-18 11:28:06.690,Fidelity,FIDELITYCOVID19,USE,100292869,100003742755,101391128,COVID-19 Home Test (PCR Lower Nasal),2020-11-18 15:26:46.320,...,1ZWV28014424426531,2020-11-18 11:28:06.690,UPS,Next Day Air,1Z550F28Y092170518,100914328.0,2020-11-18 20:46:16,2020-11-19 09:36:35,NaT,2020-11-21 10:07:30
1,100004370467_101893808,2020-12-26 09:20:36.823,www.letsgetchecked.com,None,USE,100386844,100004370467,101893808,COVID-19 Home Test (PCR Lower Nasal),2020-12-26 15:36:50.397,...,1ZWV28014432851691,2020-12-26 09:20:36.823,UPS,Next Day Air,1Z550F28Y011699832,101646453.0,2020-12-28 19:16:09,2020-12-29 14:17:21,NaT,NaT
2,100004472400_101499810,2020-12-03 15:42:43.100,BS - Yellowstone Club Pre-arrival,BSYELLOWSTONEPREARRIVALCOVID,USE,100343660,100004472400,101499810,COVID-19 (PCR Lower Nasal),2020-12-03 23:34:58.033,...,1ZWV28014439278914,2020-12-03 15:42:43.100,UPS,Next Day Air,1Z550F28Y013558141,101163535.0,2020-12-03 19:49:01,2020-12-04 12:48:55,2020-12-21 16:21:32,2020-12-22 08:33:28
3,100009629350_101489232,2020-12-03 10:51:58.227,United Health Group,UHCMEDICAREHEPC,USE,100333692,100009629350,101489232,Hep C,2020-12-07 22:41:30.120,...,9400116902881675927238,2020-12-03 10:51:58.227,UPS,Next Day Air,1Z550F28Y089964828,101629583.0,NaT,2020-12-11 10:58:00,NaT,NaT
4,100011905254_77421,2019-12-02 18:29:00.997,www.letsgetchecked.com,None,EUW,99908,100011905254,77421,Simple 2,2019-12-03 11:26:33.453,...,LG044647425IE,NaT,None,None,None,NaN,NaT,NaT,NaT,NaT


In [20]:
##to find the records not present in RDS, just in the Redshift side
dfm = red_df1_part1[~ red_df1_part1["bi_shipment_id"].isin(rds_df1_part1["bi_shipment_id"])]
dfm

,bi_shipment_id,reseller_name,order_id,order_actual_dispatch_time,order_created_at_outbound,barcoded_id,test_kit_name,shipment_id_outbound,shipment_provider_name_outbound,shipment_level_outbound,...,shipment_provider_name_inbound,shipment_level_inbound,shipment_tracking_number_inbound,shipment_pickup_time_outbound,shipment_delivery_time_outbound,shipment_pickup_time_inbound,shipment_delivery_time_inbound,payer_program_pretty_name,created_at,updated_at


In [25]:
dfm.to_csv('records_not_present_RDS_bi_shipment_20210120.csv', index=False)

## Understand basic information about the RDS dataframe

In [21]:
## to see the total of records and columns
rds_df1.shape

(1329201, 22)

In [22]:
## to see the basic information about the dataframe
rds_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1329201 entries, 0 to 793238
Data columns (total 22 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   bi_shipment_id                     1329201 non-null  object        
 1   order_created_at_outbound          1329201 non-null  datetime64[ns]
 2   reseller_name                      1329201 non-null  object        
 3   payer_program_pretty_name          725642 non-null   object        
 4   region                             1329201 non-null  object        
 5   order_id                           1329201 non-null  int64         
 6   barcoded_id                        1329201 non-null  int64         
 7   shipment_id_outbound               1329201 non-null  int64         
 8   test_kit_name                      1329201 non-null  object        
 9   order_actual_dispatch_time         1325921 non-null  datetime64[ns]
 10  shipmen

In [23]:
## to detect null values in the dataframe
rds_df1.isnull().sum()

bi_shipment_id                            0
order_created_at_outbound                 0
reseller_name                             0
payer_program_pretty_name            603559
region                                    0
order_id                                  0
barcoded_id                               0
shipment_id_outbound                      0
test_kit_name                             0
order_actual_dispatch_time             3280
shipment_provider_name_outbound           0
shipment_level_outbound                   0
shipment_tracking_number_outbound         0
order_created_at_inbound             265584
shipment_provider_name_inbound       265584
shipment_level_inbound               265584
shipment_tracking_number_inbound     265584
shipment_id_inbound                  265584
shipment_pickup_time_outbound        271020
shipment_delivery_time_outbound      324639
shipment_pickup_time_inbound         840703
shipment_delivery_time_inbound       872364
dtype: int64

## Checking the total of records between Redshift and RDS

In [24]:
## validation of the total of records between Redshift and RDS
total_records = red_df1.shape[0] == rds_df1.shape[0]
total_records

True

In [25]:
## to see the column names of the Redshift dataframe
red_df1.columns

Index(['bi_shipment_id', 'reseller_name', 'order_id',
       'order_actual_dispatch_time', 'order_created_at_outbound',
       'barcoded_id', 'test_kit_name', 'shipment_id_outbound',
       'shipment_provider_name_outbound', 'shipment_level_outbound',
       'shipment_tracking_number_outbound', 'shipment_id_inbound',
       'order_created_at_inbound', 'shipment_provider_name_inbound',
       'shipment_level_inbound', 'shipment_tracking_number_inbound',
       'shipment_pickup_time_outbound', 'shipment_delivery_time_outbound',
       'shipment_pickup_time_inbound', 'shipment_delivery_time_inbound',
       'payer_program_pretty_name', 'created_at', 'updated_at'],
      dtype='object')

In [26]:
## to see the column names of the RDS dataframe
rds_df1.columns

Index(['bi_shipment_id', 'order_created_at_outbound', 'reseller_name',
       'payer_program_pretty_name', 'region', 'order_id', 'barcoded_id',
       'shipment_id_outbound', 'test_kit_name', 'order_actual_dispatch_time',
       'shipment_provider_name_outbound', 'shipment_level_outbound',
       'shipment_tracking_number_outbound', 'order_created_at_inbound',
       'shipment_provider_name_inbound', 'shipment_level_inbound',
       'shipment_tracking_number_inbound', 'shipment_id_inbound',
       'shipment_pickup_time_outbound', 'shipment_delivery_time_outbound',
       'shipment_pickup_time_inbound', 'shipment_delivery_time_inbound'],
      dtype='object')

## Checking the columns name between RDS and Redshift

In [27]:
## to find the columns required by BI team, but not present in the DAL object in Redshift
columns_not_present_dal_object = rds_df1.columns.difference(red_df1.columns)
columns_not_present_dal_object

Index(['region'], dtype='object')

## Checking the match records, mistach and nulls

In [28]:
rds_df1_drop=rds_df1.sort_values(['bi_shipment_id']).set_index('bi_shipment_id', drop=False)
red_df1_drop=red_df1.sort_values(['bi_shipment_id']).set_index('bi_shipment_id', drop=False)

### 1- bi_shipment_id

In [29]:
diff_bi_shipment_id = True if rds_df1_drop['bi_shipment_id'].compare(red_df1_drop['bi_shipment_id']).shape[0] == 0 else False
bi_shipment_id_not_match = rds_df1_drop['bi_shipment_id'].compare(red_df1_drop['bi_shipment_id']).shape[0]

In [30]:
print("Column is the same in both tables? {}".format(diff_bi_shipment_id))
print("There are {} records that don't match".format(bi_shipment_id_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['bi_shipment_id'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['bi_shipment_id'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 2- reseller_name 

In [31]:
diff_reseller_name = True if rds_df1_drop['reseller_name'].compare(red_df1_drop['reseller_name']).shape[0] == 0 else False
reseller_name_not_match = rds_df1_drop['reseller_name'].compare(red_df1_drop['reseller_name']).shape[0]

In [32]:
print("Column is the same in both tables? {}".format(diff_reseller_name))
print("There are {} records that don't match".format(reseller_name_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['reseller_name'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['reseller_name'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 3- payer_program_pretty_name

In [33]:
diff_payer_program_pretty_name = True if rds_df1_drop['payer_program_pretty_name'].compare(red_df1_drop['payer_program_pretty_name']).shape[0] == 0 else False
payer_program_pretty_name_not_match = rds_df1_drop['payer_program_pretty_name'].compare(red_df1_drop['payer_program_pretty_name']).shape[0]

In [34]:
print("Column is the same in both tables? {}".format(diff_payer_program_pretty_name))
print("There are {} records that don't match".format(payer_program_pretty_name_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['payer_program_pretty_name'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['payer_program_pretty_name'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 603559 null values
Null values in Redshift: 603559 null values


### 4- order_id

In [35]:
diff_order_id = True if rds_df1_drop['order_id'].compare(red_df1_drop['order_id']).shape[0] == 0 else False
order_id_not_match = rds_df1_drop['order_id'].compare(red_df1_drop['order_id']).shape[0]

In [36]:
print("Column is the same in both tables? {}".format(diff_order_id))
print("There are {} records that don't match".format(order_id_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['order_id'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['order_id'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 5- barcoded_id

In [37]:
diff_barcoded_id = True if rds_df1_drop['barcoded_id'].compare(red_df1_drop['barcoded_id']).shape[0] == 0 else False
barcoded_id_not_match = rds_df1_drop['barcoded_id'].compare(red_df1_drop['barcoded_id']).shape[0]

In [38]:
print("Column is the same in both tables? {}".format(diff_barcoded_id))
print("There are {} records that don't match".format(barcoded_id_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['barcoded_id'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['barcoded_id'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 6- test_kit_name

In [39]:
diff_test_kit_name = True if rds_df1_drop['test_kit_name'].compare(red_df1_drop['test_kit_name']).shape[0] == 0 else False
test_kit_name_not_match = rds_df1_drop['test_kit_name'].compare(red_df1_drop['test_kit_name']).shape[0]

In [40]:
print("Column is the same in both tables? {}".format(diff_test_kit_name))
print("There are {} records that don't match".format(test_kit_name_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['test_kit_name'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['test_kit_name'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 7- order_actual_dispatch_time

In [41]:
diff_order_actual_dispatch_time = True if rds_df1_drop['order_actual_dispatch_time'].compare(red_df1_drop['order_actual_dispatch_time']).shape[0] == 0 else False
order_actual_dispatch_time_not_match = rds_df1_drop['order_actual_dispatch_time'].compare(red_df1_drop['order_actual_dispatch_time']).shape[0]

In [42]:
print("Column is the same in both tables? {}".format(diff_order_actual_dispatch_time))
print("There are {} records that don't match".format(order_actual_dispatch_time_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['order_actual_dispatch_time'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['order_actual_dispatch_time'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 3280 null values
Null values in Redshift: 3280 null values


### 8- shipment_id_outbound

In [43]:
diff_shipment_id_outbound = True if rds_df1_drop['shipment_id_outbound'].compare(red_df1_drop['shipment_id_outbound']).shape[0] == 0 else False
shipment_id_outbound_not_match = rds_df1_drop['shipment_id_outbound'].compare(red_df1_drop['shipment_id_outbound']).shape[0]

In [44]:
print("Column is the same in both tables? {}".format(diff_shipment_id_outbound))
print("There are {} records that don't match".format(shipment_id_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_id_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_id_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 9- shipment_provider_name_outbound

In [45]:
diff_shipment_provider_name_outbound = True if rds_df1_drop['shipment_provider_name_outbound'].compare(red_df1_drop['shipment_provider_name_outbound']).shape[0] == 0 else False
shipment_provider_name_outbound_not_match = rds_df1_drop['shipment_provider_name_outbound'].compare(red_df1_drop['shipment_provider_name_outbound']).shape[0]

In [46]:
print("Column is the same in both tables? {}".format(diff_shipment_provider_name_outbound))
print("There are {} records that don't match".format(shipment_provider_name_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_provider_name_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_provider_name_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 10- shipment_level_outbound

In [47]:
diff_shipment_level_outbound = True if rds_df1_drop['shipment_level_outbound'].compare(red_df1_drop['shipment_level_outbound']).shape[0] == 0 else False
shipment_level_outbound_not_match = rds_df1_drop['shipment_level_outbound'].compare(red_df1_drop['shipment_level_outbound']).shape[0]

In [48]:
print("Column is the same in both tables? {}".format(diff_shipment_level_outbound))
print("There are {} records that don't match".format(shipment_level_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_level_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_level_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 11- shipment_tracking_number_outbound

In [49]:
diff_shipment_tracking_number_outbound = True if rds_df1_drop['shipment_tracking_number_outbound'].compare(red_df1_drop['shipment_tracking_number_outbound']).shape[0] == 0 else False
shipment_tracking_number_outbound_not_match = rds_df1_drop['shipment_tracking_number_outbound'].compare(red_df1_drop['shipment_tracking_number_outbound']).shape[0]

In [50]:
print("Column is the same in both tables? {}".format(diff_shipment_tracking_number_outbound))
print("There are {} records that don't match".format(shipment_tracking_number_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_tracking_number_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_tracking_number_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 12- shipment_id_inbound

In [51]:
diff_shipment_id_inbound = True if rds_df1_drop['shipment_id_inbound'].compare(red_df1_drop['shipment_id_inbound']).shape[0] == 0 else False
shipment_id_inbound_not_match = rds_df1_drop['shipment_id_inbound'].compare(red_df1_drop['shipment_id_inbound']).shape[0]

In [52]:
print("Column is the same in both tables? {}".format(diff_shipment_id_inbound))
print("There are {} records that don't match".format(shipment_id_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_id_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_id_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 1 records that don't match
Null values in SQL: 265584 null values
Null values in Redshift: 265585 null values


In [53]:
diff_shipment_id_inbound = rds_df1_drop['shipment_id_inbound'].compare(red_df1_drop['shipment_id_inbound'])
diff_shipment_id_inbound.head()

,self,other
bi_shipment_id,,
51801451928_104703744,104717644.0,NaN


### 13- shipment_provider_name_inbound

In [54]:
diff_shipment_provider_name_inbound = True if rds_df1_drop['shipment_provider_name_inbound'].compare(red_df1_drop['shipment_provider_name_inbound']).shape[0] == 0 else False
shipment_provider_name_inbound_not_match = rds_df1_drop['shipment_provider_name_inbound'].compare(red_df1_drop['shipment_provider_name_inbound']).shape[0]

In [55]:
print("Column is the same in both tables? {}".format(diff_shipment_provider_name_inbound))
print("There are {} records that don't match".format(shipment_provider_name_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_provider_name_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_provider_name_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 1 records that don't match
Null values in SQL: 265584 null values
Null values in Redshift: 265585 null values


In [78]:
diff_shipment_provider_name_inbound = rds_df1_drop['shipment_provider_name_inbound'].compare(red_df1_drop['shipment_provider_name_inbound'])
diff_shipment_provider_name_inbound.head()

,self,other
bi_shipment_id,,
51801451928_104703744,UPS,None


### 14- shipment_level_inbound

In [57]:
diff_shipment_level_inbound = True if rds_df1_drop['shipment_level_inbound'].compare(red_df1_drop['shipment_level_inbound']).shape[0] == 0 else False
shipment_level_inbound_not_match = rds_df1_drop['shipment_level_inbound'].compare(red_df1_drop['shipment_level_inbound']).shape[0]

In [58]:
print("Column is the same in both tables? {}".format(diff_shipment_level_inbound))
print("There are {} records that don't match".format(shipment_level_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_level_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_level_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 1 records that don't match
Null values in SQL: 265584 null values
Null values in Redshift: 265585 null values


In [59]:
diff_shipment_level_inbound = rds_df1_drop['shipment_level_inbound'].compare(red_df1_drop['shipment_level_inbound'])
diff_shipment_level_inbound.head()

,self,other
bi_shipment_id,,
51801451928_104703744,UPS Next Day Air,None


### 15- shipment_tracking_number_inbound

In [60]:
diff_shipment_tracking_number_inbound = True if rds_df1_drop['shipment_tracking_number_inbound'].compare(red_df1_drop['shipment_tracking_number_inbound']).shape[0] == 0 else False
shipment_tracking_number_inbound_not_match = rds_df1_drop['shipment_tracking_number_inbound'].compare(red_df1_drop['shipment_tracking_number_inbound']).shape[0]

In [61]:
print("Column is the same in both tables? {}".format(diff_shipment_tracking_number_inbound))
print("There are {} records that don't match".format(shipment_tracking_number_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_tracking_number_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_tracking_number_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 1 records that don't match
Null values in SQL: 265584 null values
Null values in Redshift: 265585 null values


In [62]:
diff_shipment_tracking_number_inbound = rds_df1_drop['shipment_tracking_number_inbound'].compare(red_df1_drop['shipment_tracking_number_inbound'])
diff_shipment_tracking_number_inbound.head()

,self,other
bi_shipment_id,,
51801451928_104703744,1ZR866X7Y008608226,None


### 16- shipment_pickup_time_outbound

In [63]:
diff_shipment_pickup_time_outbound = True if rds_df1_drop['shipment_pickup_time_outbound'].compare(red_df1_drop['shipment_pickup_time_outbound']).shape[0] == 0 else False
shipment_pickup_time_outbound_not_match = rds_df1_drop['shipment_pickup_time_outbound'].compare(red_df1_drop['shipment_pickup_time_outbound']).shape[0]

In [64]:
print("Column is the same in both tables? {}".format(diff_shipment_pickup_time_outbound))
print("There are {} records that don't match".format(shipment_pickup_time_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_pickup_time_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_pickup_time_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 271020 null values
Null values in Redshift: 271020 null values


### 17- shipment_delivery_time_outbound

In [65]:
diff_shipment_delivery_time_outbound = True if rds_df1_drop['shipment_delivery_time_outbound'].compare(red_df1_drop['shipment_delivery_time_outbound']).shape[0] == 0 else False
shipment_delivery_time_outbound_not_match = rds_df1_drop['shipment_delivery_time_outbound'].compare(red_df1_drop['shipment_delivery_time_outbound']).shape[0]

In [66]:
print("Column is the same in both tables? {}".format(diff_shipment_delivery_time_outbound))
print("There are {} records that don't match".format(shipment_delivery_time_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_delivery_time_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_delivery_time_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 324639 null values
Null values in Redshift: 324639 null values


### 18- shipment_pickup_time_inbound

In [67]:
diff_shipment_pickup_time_inbound = True if rds_df1_drop['shipment_pickup_time_inbound'].compare(red_df1_drop['shipment_pickup_time_inbound']).shape[0] == 0 else False
shipment_pickup_time_inbound_not_match = rds_df1_drop['shipment_pickup_time_inbound'].compare(red_df1_drop['shipment_pickup_time_inbound']).shape[0]

In [68]:
print("Column is the same in both tables? {}".format(diff_shipment_pickup_time_inbound))
print("There are {} records that don't match".format(shipment_pickup_time_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_pickup_time_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_pickup_time_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 526 records that don't match
Null values in SQL: 840703 null values
Null values in Redshift: 841229 null values


In [69]:
diff_shipment_pickup_time_inbound = rds_df1_drop['shipment_pickup_time_inbound'].compare(red_df1_drop['shipment_pickup_time_inbound'])
diff_shipment_pickup_time_inbound.head()

,self,other
bi_shipment_id,,
106806610402_104813355,2022-01-20 18:58:53,NaT
114060862395_104663912,2022-01-20 18:58:53,NaT
114970909296_104585626,2022-01-18 23:23:57,NaT
116452950273_104686885,2022-01-20 19:11:26,NaT
117607334386_104874845,2022-01-20 18:46:42,NaT


### 19- shipment_delivery_time_inbound

In [70]:
diff_shipment_delivery_time_inbound = True if rds_df1_drop['shipment_delivery_time_inbound'].compare(red_df1_drop['shipment_delivery_time_inbound']).shape[0] == 0 else False
shipment_delivery_time_inbound_not_match = rds_df1_drop['shipment_delivery_time_inbound'].compare(red_df1_drop['shipment_delivery_time_inbound']).shape[0]

In [71]:
print("Column is the same in both tables? {}".format(diff_shipment_delivery_time_inbound))
print("There are {} records that don't match".format(shipment_delivery_time_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['shipment_delivery_time_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['shipment_delivery_time_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 133 records that don't match
Null values in SQL: 872364 null values
Null values in Redshift: 872497 null values


In [72]:
diff_shipment_delivery_time_inbound = rds_df1_drop['shipment_delivery_time_inbound'].compare(red_df1_drop['shipment_delivery_time_inbound'])
diff_shipment_delivery_time_inbound.head()

,self,other
bi_shipment_id,,
102921255159_104583071,2022-01-20 09:36:00,NaT
107534760737_104693564,2022-01-20 09:36:00,NaT
110127388443_104584699,2022-01-20 09:36:00,NaT
114970909296_104585626,2022-01-20 09:36:00,NaT
126764492683_104706918,2022-01-20 09:36:00,NaT


### 20- order_created_at_outbound

In [73]:
diff_order_created_at_outbound = True if rds_df1_drop['order_created_at_outbound'].compare(red_df1_drop['order_created_at_outbound']).shape[0] == 0 else False
order_created_at_outbound_not_match = rds_df1_drop['order_created_at_outbound'].compare(red_df1_drop['order_created_at_outbound']).shape[0]

In [74]:
print("Column is the same in both tables? {}".format(diff_order_created_at_outbound))
print("There are {} records that don't match".format(order_created_at_outbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['order_created_at_outbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['order_created_at_outbound'].isnull().sum()))

Column is the same in both tables? True
There are 0 records that don't match
Null values in SQL: 0 null values
Null values in Redshift: 0 null values


### 21- order_created_at_inbound

In [75]:
diff_order_created_at_inbound = True if rds_df1_drop['order_created_at_inbound'].compare(red_df1_drop['order_created_at_inbound']).shape[0] == 0 else False
order_created_at_inbound_not_match = rds_df1_drop['order_created_at_inbound'].compare(red_df1_drop['order_created_at_inbound']).shape[0]

In [76]:
print("Column is the same in both tables? {}".format(diff_order_created_at_inbound))
print("There are {} records that don't match".format(order_created_at_inbound_not_match))
print("Null values in SQL: {} null values".format(rds_df1_drop['order_created_at_inbound'].isnull().sum()))
print("Null values in Redshift: {} null values".format(red_df1_drop['order_created_at_inbound'].isnull().sum()))

Column is the same in both tables? False
There are 1 records that don't match
Null values in SQL: 265584 null values
Null values in Redshift: 265585 null values


In [77]:
diff_order_created_at_inbound = rds_df1_drop['order_created_at_inbound'].compare(red_df1_drop['order_created_at_inbound'])
diff_order_created_at_inbound.head()

,self,other
bi_shipment_id,,
51801451928_104703744,2022-01-11 20:37:57.907,NaT
